In [ ]:
%pip install fairseq tensorboardX
%pip install https://github.com/RobotsMali-AI/rmai/releases/download/0.0.4/rmaipkg-0.0.4.tar.gz
%pip install datasets OpenNMT-py==3.0.0 sentencepiece

In [ ]:
!git clone https://github.com/microsoft/unilm.git
%cd unilm/deltalm
!git submodule update --init fairseq
!git clone https://github.com/Gjuuuy/MT_base.git
%cd deltalm/models
!wget https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model
!wget https://dl.fbaipublicfiles.com/large_objects/nllb/models/spm_200/dictionary.txt

In [ ]:
%cd ../..

In [ ]:
#installing spm commands
!git clone https://github.com/google/sentencepiece.git 
%cd sentencepiece
%mkdir build
%cd build
!cmake ..
!make -j $(nproc)
!sudo make install
!sudo ldconfig -v

In [ ]:
%cd ../..

In [ ]:
%mkdir MT_base/spm_data

In [ ]:
!spm_encode --model=deltalm/models/flores200_sacrebleu_tokenizer_spm.model --output_format=piece < MT_base/data/train.bam > MT_base/spm_data/train.spm.bam
!spm_encode --model=deltalm/models/flores200_sacrebleu_tokenizer_spm.model --output_format=piece < MT_base/data/train.fr > MT_base/spm_data/train.spm.fr
!spm_encode --model=deltalm/models/flores200_sacrebleu_tokenizer_spm.model --output_format=piece < MT_base/data/dev.bam > MT_base/spm_data/dev.spm.bam
!spm_encode --model=deltalm/models/flores200_sacrebleu_tokenizer_spm.model --output_format=piece < MT_base/data/dev.fr > MT_base/spm_data/dev.spm.fr
!spm_encode --model=deltalm/models/flores200_sacrebleu_tokenizer_spm.model --output_format=piece < MT_base/data/test.bam > MT_base/spm_data/test.spm.bam
!spm_encode --model=deltalm/models/flores200_sacrebleu_tokenizer_spm.model --output_format=piece < MT_base/data/test.fr > MT_base/spm_data/test.spm.fr

In [ ]:
!mkdir MT_base/data_bin
!python preprocess.py  \
    --trainpref MT_base/spm_data/train.spm \
    --validpref MT_base/spm_data/dev.spm \
    --testpref MT_base/spm_data/test.spm \
    --source-lang bam --target-lang fr \
    --destdir MT_base/data_bin \
    --srcdict deltalm/models/dictionary.txt \
    --tgtdict deltalm/models/dictionary.txt \
    --workers 40

In [ ]:
!wget https://s3.amazonaws.com/opennmt-models/nllb-200/nllb-200-3.3B-onmt.pt -P MT_base/

In [ ]:
!PRETRAINED_MODEL='MT_base/nllb-200-3.3B-onmt.pt'
!save_dir='MT_base/bam2fr'
!bin_dir='MT_base/data_bin'
!batch_size=4096
!lr=2

!mkdir MT_base/bam2fr

!python train.py MT_base/data_bin --save-dir MT_base/bam2fr --arch deltalm_large \
    --pretrained-deltalm-checkpoint MT_base/nllb-200-3.3B-onmt.pt \
    --share-all-embeddings \
    --max-source-positions 512 --max-target-positions 512 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' \
    --lr-scheduler inverse_sqrt \
    --lr 2 \
    --warmup-init-lr 1e-07 \
    --stop-min-lr 1e-09 \
    --warmup-updates 200 \
    --max-update 1000 \
    --max-epoch 100 \
    --max-tokens 4096 \
    --update-freq 1 \
    --seed 1 \
    --log-format simple \
    --skip-invalid-size-inputs-valid-test

mkdir: cannot create directory ‘MT_base/bam2fr’: File exists
2023-02-14 03:50:22 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view':

In [ ]:
python generate.py MT_base/data_bin \
    --path MT_base/bam2fr/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe=sentencepiece